# Example 14: Hyperparameter Optimization with Ray Tune

Manually tuning hyperparameters -- learning rate, hidden size, model type --
is tedious and error-prone. TSFast integrates with
[Ray Tune](https://docs.ray.io/en/latest/tune/index.html) to automate the
search. This example runs a small hyperparameter search to find the best
model configuration for the Silverbox benchmark.

## Prerequisites

This example builds on concepts from:

- **Example 00** -- data loading and model training basics
- **Example 04** -- model architectures and `rnn_type`

Make sure Ray Tune is installed:

```bash
uv sync --extra dev
```

## Setup

In [1]:
from tsfast.datasets.benchmark import create_dls_silverbox
from tsfast.models.rnn import RNNLearner
from tsfast.tune import HPOptimizer, log_uniform
from tsfast.learner.losses import fun_rmse
from ray import tune

/home/pheenix/Development/tsfast/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2026-02-22 21:03:56,436	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


2026-02-22 21:03:56,652	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Why Hyperparameter Optimization?

Model performance depends heavily on hyperparameters: learning rate, hidden
size, architecture choice, and regularization strength. Finding the right
combination by hand requires many experiments and careful record-keeping.

Automated approaches help:

- **Grid search** evaluates every combination -- thorough but expensive.
- **Random search** samples randomly and is surprisingly effective in
  high-dimensional spaces.
- **Population-based training** evolves configurations during training,
  combining exploration with exploitation.

Ray Tune provides all of these strategies (and more) behind a unified API.
TSFast's `HPOptimizer` wraps Ray Tune so you can search over model
configurations with minimal boilerplate.

## Prepare the DataLoaders

We use the Silverbox benchmark with a small batch size and window size to
keep the example lightweight.

In [2]:
dls = create_dls_silverbox(bs=16, win_sz=500, stp_sz=10)

## Define a Learner Factory

`HPOptimizer` needs a factory function that takes `(dls, config)` and returns
a configured Learner. Ray Tune calls this function once per trial, each time
with a different hyperparameter configuration sampled from the search space.

In [3]:
def create_learner(dls, config):
    """Create a configured RNNLearner from hyperparameter config."""
    return RNNLearner(
        dls,
        rnn_type=config["rnn_type"],
        hidden_size=config["hidden_size"],
        n_skip=50,
        metrics=[fun_rmse],
    )

## Define the Search Space

The search space is a plain dictionary where values are Ray Tune sampling
primitives:

- **`tune.choice`** -- samples uniformly from a list of discrete options.
  Good for categorical parameters like architecture type or layer count.
- **`log_uniform`** -- samples uniformly on a logarithmic scale. Ideal for
  parameters that span orders of magnitude, such as learning rate.

We start with a small search over two parameters: RNN cell type and hidden
size.

In [4]:
search_config = {
    "rnn_type": tune.choice(["gru", "lstm"]),
    "hidden_size": tune.choice([32, 40]),
    "n_epoch": 3,
    "lr": 3e-3,
}

The config also contains fixed training parameters:

- **`n_epoch=3`** -- each trial trains for 3 epochs (enough to compare
  configurations, not enough for final training).
- **`lr=3e-3`** -- fixed learning rate for all trials in this first search.

## Run the Optimization

`HPOptimizer` takes the learner factory and the DataLoaders. Calling
`optimize` launches the search: `num_samples=4` runs 4 independent trials,
each with a different hyperparameter combination drawn from `search_config`.

The default training function uses `fit_flat_cos` and reports training loss,
validation loss, and metrics to Ray Tune after every epoch.

In [5]:
optimizer = HPOptimizer(
    create_lrn=create_learner,
    dls=dls,
)

results = optimizer.optimize(
    config=search_config,
    num_samples=4,
    resources_per_trial={"cpu": 1, "gpu": 0},
)

(learner_optimize pid=34385) /home/pheenix/Development/tsfast/.venv/lib/python3.12/site-packages/fastai/torch_core.py:139: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:213.)
(learner_optimize pid=34385)   t = torch.as_tensor(x, **kwargs)


(learner_optimize pid=34385) [0, 0.006426369305700064, 0.003564209211617708, 0.0048643071204423904, '00:05']


(learner_optimize pid=34385) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/pheenix/ray_results/learner_optimize_2026-02-22_21-04-00/learner_optimize_a1008_00001_1_hidden_size=32,rnn_type=lstm_2026-02-22_21-04-00/checkpoint_000000)
(learner_optimize pid=34383) /home/pheenix/Development/tsfast/.venv/lib/python3.12/site-packages/fastai/torch_core.py:139: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:213.) [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-log

(learner_optimize pid=34386) [0, 0.004843713715672493, 0.003075819229707122, 0.004263146780431271, '00:06']


(learner_optimize pid=34385) [1, 0.0024192084092646837, 0.002331544877961278, 0.0032735918648540974, '00:05']


...

(learner_optimize pid=34384) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/pheenix/ray_results/learner_optimize_2026-02-22_21-04-00/learner_optimize_a1008_00002_2_hidden_size=32,rnn_type=gru_2026-02-22_21-04-00/checkpoint_000002) [repeated 2x across cluster]


(learner_optimize pid=34384) [2, 0.0010295608080923557, 0.0010033256839960814, 0.0021213546860963106, '00:36']


2026-02-22 21:06:07,078	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/pheenix/ray_results/learner_optimize_2026-02-22_21-04-00' in 0.0048s.


(learner_optimize pid=34383) [2, 0.0010143726831302047, 0.0009642408112995327, 0.002002180088311434, '00:35']


2026-02-22 21:06:07,083	INFO tune.py:1041 -- Total run time: 126.49 seconds (126.44 seconds for the tuning loop).


## Analyze Results

The `optimize` call returns a Ray Tune `ExperimentAnalysis` object stored in
`optimizer.analysis`. You can query it for the best trial configuration,
inspect per-trial results, or export data for further analysis.

In [6]:
best = optimizer.analysis.get_best_config(metric="valid_loss", mode="min")
print("Best config:")
for key in ["rnn_type", "hidden_size", "lr"]:
    print(f"  {key}: {best[key]}")

Best config:
  rnn_type: gru
  hidden_size: 32
  lr: 0.003


In [7]:
result_df = optimizer.analysis.results_df
print("\nAll trial results:")
result_df[["config/rnn_type", "config/hidden_size", "valid_loss"]]


All trial results:


,config/rnn_type,config/hidden_size,valid_loss
trial_id,,,
a1008_00000,gru,32,0.000964
a1008_00001,lstm,32,0.001087
a1008_00002,gru,32,0.001003
a1008_00003,lstm,40,0.001099


## Using log_uniform for Learning Rate

In the first search we fixed the learning rate. A more thorough search treats
`lr` as a tunable parameter using `log_uniform`. This samples on a
logarithmic scale between the given bounds -- appropriate because the
difference between `1e-4` and `1e-3` matters more than between `1e-2` and
`1.1e-2`.

In [8]:
search_config_v2 = {
    "rnn_type": tune.choice(["gru", "lstm"]),
    "hidden_size": tune.choice([32, 40]),
    "lr": log_uniform(1e-4, 1e-2),
    "n_epoch": 3,
}

When `lr` is a callable sampler in the config, the training function samples
a fresh value for each trial. This overrides any fixed learning rate.

In [9]:
optimizer_v2 = HPOptimizer(
    create_lrn=create_learner,
    dls=dls,
)

results_v2 = optimizer_v2.optimize(
    config=search_config_v2,
    num_samples=4,
    resources_per_trial={"cpu": 1, "gpu": 0},
)

2026-02-22 21:07:48,421	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/pheenix/ray_results/learner_optimize_2026-02-22_21-06-07' in 0.0075s.


2026-02-22 21:07:48,424	INFO tune.py:1041 -- Total run time: 101.30 seconds (101.28 seconds for the tuning loop).


In [10]:
best_v2 = optimizer_v2.analysis.get_best_config(metric="valid_loss", mode="min")
print("Best config (with lr search):")
for key in ["rnn_type", "hidden_size", "lr"]:
    print(f"  {key}: {best_v2[key]}")

Best config (with lr search):
  rnn_type: gru
  hidden_size: 32
  lr: <function log_uniform.<locals>._sample at 0x75c0dc079300>


## Key Takeaways

- **`HPOptimizer`** wraps Ray Tune for easy hyperparameter search with
  TSFast. Pass a learner factory and DataLoaders, then call `optimize`.
- **Learner factory** -- a function `(dls, config) -> Learner` that builds a
  fresh model from the hyperparameter config each trial.
- **`tune.choice`** for categorical parameters (architecture, layer count);
  **`log_uniform`** for continuous parameters on a log scale (learning rate).
- **Start small** -- few trials, few epochs -- to validate the pipeline
  before scaling up.
- **`optimizer.analysis`** gives access to the full Ray Tune
  `ExperimentAnalysis` for querying best configs, exporting results, and
  loading the best checkpoint.